# Using Swig

## fonctions with numpy list parameter

http://swig.org/Doc1.1/HTML/Typemaps.html
https://docs.scipy.org/doc/numpy/reference/swig.interface-file.html
https://scipy.github.io/old-wiki/pages/Cookbook/SWIG_NumPy_examples.html


### C file array_sum.c

In [ ]:
float array_sum(float* tab, unsigned int size){

    float result = 0.;
        for ( unsigned int i =0; i < size; ++i){
            result += tab[i];
        }
        return result;
}

     

Now, in order to add these files to your favorite language, you need to write an "interface file" which is the input to SWIG. An interface file for these C functions might look like this :

### swig interface file modarray.i

In [2]:
%module modarray

%{
    #define SWIG_FILE_WITH_INIT
    #include "array_sum.h"
%}

%include "numpy.i"

%init %{
    import_array();
%}

%apply (float* IN_ARRAY1, int DIM1) {(float* tab, int n)}

%include "array_sum.h"


IndentationError: unexpected indent (<ipython-input-2-28a17a7d21cf>, line 11)

The %apply directive tells SWIG that we are going to apply special processing to a datatype. The "double *OUTPUT" is the name of a rule describing how to return an output value from a "double *" (this rule is defined in the file typemaps.i). The rule gets applied to all of the datatypes listed in curly braces-- in this case "double *result".

https://scipy.github.io/old-wiki/pages/Cookbook/SWIG_NumPy_examples.html
ARGOUT_ARRAY1
INPLACE_ARRAY1

In [14]:
! swig -python modarray.i

In [15]:
! gcc -fPIC -c array_sum.c modarray_wrap.c -I/home/jacquem/anaconda3/include/python3.5m
! gcc -shared array_sum.o modarray_wrap.o -o _modarray.so

In [20]:
import modarray
import numpy as np

In [22]:
l = np.arange(100, dtype=np.float32)
modarray.array_sum(l)

4950.0

In [23]:
l = [1,2,3,4,5,6,7,8]
modarray.array_sum(l)

36.0

In [24]:
l=(1,2,3,4,5)
modarray.array_sum(l)

15.0